In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [19]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
%matplotlib inline

In [72]:
df = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/df.pkl')

To start, the main way I'll be adding features here is by making lag and rolling features. I also want to correct for that fact that not all data in each row is knowable _before_ the game. I know the fixture and the opponent before a game, but I don't know the minutes a player played until after, for example

In [73]:
prior_knowledge = ['player','team', 'position', 'gw', 'opponent_team', 'was_home', 'kickoff_time', 'season']
posterior_knowledge = ['team_h_score', 'team_a_score', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 
                       'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
                      'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index',
                      'transfers_balance', 'selected', 'transfers_in', 'transfers_out']

In [74]:
#ordering the columns so I can keep better track of this
df = df[['total_points'] + prior_knowledge + posterior_knowledge]

In [76]:
df = df.sort_values(['player', 'kickoff_time'])
shifted_df = df.groupby('player')[['total_points'] + posterior_knowledge].shift(1)
df = df.join(shifted_df, rsuffix='_1').sort_values(['player', 'kickoff_time'])

In [85]:
#use the shifted columns (with suffix '_1' to calculate rolling windows)

df = df.sort_values(['player', 'kickoff_time'])
df['rolling_total_points'] = df.groupby('player')['total_points_1'].rolling(window=3, min_periods=3).mean().values